Multinomial Logistic Regression - HeartDiseasePredictor

In [1]:
#importing dependencies 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
col_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'] #column names

df = pd.read_csv('processed.cleveland.data.csv', header = None)
df.columns = col_names # setting dataframe column names
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


Data Processing 

In [3]:
for i in df.columns:
    print(df[i].value_counts())

age
58.0    19
57.0    17
54.0    16
59.0    14
52.0    13
60.0    12
51.0    12
56.0    11
62.0    11
44.0    11
64.0    10
41.0    10
67.0     9
63.0     9
42.0     8
43.0     8
45.0     8
53.0     8
55.0     8
61.0     8
65.0     8
50.0     7
66.0     7
48.0     7
46.0     7
47.0     5
49.0     5
70.0     4
68.0     4
35.0     4
39.0     4
69.0     3
71.0     3
40.0     3
34.0     2
37.0     2
38.0     2
29.0     1
77.0     1
74.0     1
76.0     1
Name: count, dtype: int64
sex
1.0    206
0.0     97
Name: count, dtype: int64
cp
4.0    144
3.0     86
2.0     50
1.0     23
Name: count, dtype: int64
trestbps
120.0    37
130.0    36
140.0    32
110.0    19
150.0    17
138.0    12
128.0    12
160.0    11
125.0    11
112.0     9
132.0     8
118.0     7
124.0     6
108.0     6
135.0     6
152.0     5
134.0     5
145.0     5
100.0     4
170.0     4
122.0     4
126.0     3
136.0     3
115.0     3
180.0     3
142.0     3
105.0     3
102.0     2
146.0     2
144.0     2
148.0     2
178.0     2
9

In [4]:
df.replace({'?': np.nan}, inplace = True) # converting '?' to NaN values
df[['ca', 'thal']] = df[['ca', 'thal']].astype('float64') # Casting columns data-type to floats
df['ca'].replace({np.nan: df['ca'].median()}, inplace = True) # replaces null values of ca column with median value
df['thal'].replace({np.nan: df['thal'].median()}, inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    float64
 12  thal      303 non-null    float64
 13  num       303 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 33.3 KB


C:\Users\user\AppData\Local\Temp\ipykernel_5140\2639729781.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ca'].replace({np.nan: df['ca'].median()}, inplace = True) # replaces null values of ca column with median value
C:\Users\user\AppData\Local\Temp\ipykernel_5140\2639729781.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

In [5]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.663366,4.722772,0.937294
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.934375,1.938383,1.228536
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,2.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,4.000000


In [6]:
# selecting all the features within our dataset
features = df[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']] 
features = features.to_numpy() # converts feature set to numpy array
target = df['num'].to_numpy() # converts target column to numpy array
features.shape, len(target) 

((303, 13), 303)

In [7]:
# function for standardizing data
def standardScaler(feature_array):
    """Takes the numpy.ndarray object containing the features and performs standardization on the matrix.
    The function iterates through each column and performs scaling on them individually.
    
    Args-
        feature_array- Numpy array containing training features
    
    Returns-
        None
    """
    
    total_cols = feature_array.shape[1] # total number of columns 
    for i in range(total_cols): # iterating through each column
        feature_col = feature_array[:, i]
        mean = feature_col.mean() # mean stores mean value for the column
        std = feature_col.std() # std stores standard deviation value for the column
        feature_array[:, i] = (feature_array[:, i] - mean) / std # standard scaling of each element of the column

In [8]:
standardScaler(features) # performing standardization on our feature set 

# checking if standardization worked
total_cols = features.shape[1] # total number of columns 
for i in range(total_cols):
    print(features[:, i].std())

0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999998
1.0
1.0000000000000002
1.0
0.9999999999999999


Formulating the model 

In [9]:
weights = np.random.rand(5,13)
biases = np.random.rand(5,1)

In [10]:
def linearPredict(featureMat, weights, biases):
    """This is the linear predictor function for out MLR model. It calculates the logit scores for each possible outcome.
    
    Args-
        featureMat- A numpy array of features
        weights- A numpy array of weights for our model
        biases- A numpy array of biases for our model
    
    Returns-
        logitScores- Logit scores for each possible outcome of the target variable for each feature set in the feature matrix
    """
    logitScores = np.array([np.empty([5]) for i in range(featureMat.shape[0])]) # creating empty(garbage value) array for each feature set
    
    for i in range(featureMat.shape[0]): # iterating through each feature set
        logitScores[i] = (weights.dot(featureMat[i].reshape(-1,1)) + biases).reshape(-1) # calculates logit score for each feature set then flattens the logit vector 
    
    return logitScores

In [17]:
featuresTest = df[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
featuresTest = featuresTest.to_numpy() # converts feature set to numpy array
logitTest = linearPredict(featuresTest, weights, biases)
logitTest.shape

(303, 5)

In [12]:
logitTest[0]

array([474.4334752 , 460.16184832, 273.43674625, 376.76984403,
       293.05525386])

In [13]:
def softmaxNormalizer(logitMatrix):
    """Converts logit scores for each possible outcome to probability values.
    
    Args-
        logitMatrix - This is the output of our logitPredict function; consists  logit scores for each feature set
    
    Returns-
        probabilities - Probability value of each outcome for each feature set
    """
    
    probabilities = np.array([np.empty([5]) for i in range(logitMatrix.shape[0])]) # creating empty(garbage value) array for each feature set

    for i in range(logitMatrix.shape[0]):
        exp = np.exp(logitMatrix[i]) # exponentiates each element of the logit array
        sumOfArr = np.sum(exp) # adds up all the values in the exponentiated array
        probabilities[i] = exp/sumOfArr # logit scores to probability values
    return probabilities

In [14]:
def multinomialLogReg(features, weights, biases):
    """Performs logistic regression on a given feature set.
    
    Args- 
        features- Numpy array of features(standardized)
        weights- A numpy array of weights for our model
        biases- A numpy array of biases for our model
    
    Returns-
        probabilities, predictions
        Here,
            probabilities: Probability values for each possible outcome for each feature set in the feature matrix
            predictions: Outcome with max probability for each feature set
    """
    logitScores = linearPredict(features, weights, biases)
    probabilities = softmaxNormalizer(logitScores) 
    predictions = np.array([np.argmax(i) for i in probabilities]) #returns the outcome with max probability
    return probabilities, predictions

In [15]:
probabilities,predictions = multinomialLogReg(features, weights, biases)
print(probabilities.shape)
print(predictions)

(303, 5)
[1 2 2 1 3 4 1 3 2 0 2 1 0 0 0 3 1 4 3 4 1 1 1 2 2 1 4 1 3 2 1 3 1 2 3 4 2
 2 3 1 2 3 1 1 4 2 4 2 1 0 4 2 4 4 3 2 2 4 2 4 3 3 2 0 2 2 1 0 1 1 1 0 2 2
 4 1 2 3 0 2 4 4 4 1 4 4 4 4 3 3 4 1 2 4 1 3 2 2 4 4 4 4 4 1 2 4 3 2 1 4 1
 0 0 3 1 4 0 3 2 2 0 1 4 1 1 1 1 3 4 3 2 4 4 0 3 1 1 2 3 0 3 1 0 1 2 4 2 4
 4 4 1 4 1 2 2 2 3 2 2 1 0 4 1 4 4 3 4 0 4 4 1 2 3 1 2 2 4 2 4 4 2 3 4 1 3
 3 0 2 0 2 4 3 0 1 1 4 1 3 3 1 4 3 0 1 4 2 2 2 4 3 3 1 4 3 4 1 4 3 1 0 3 4
 4 2 3 3 4 3 4 4 4 1 4 1 3 4 2 4 1 4 4 3 3 1 1 2 4 4 3 0 4 2 2 4 4 4 4 1 1
 1 4 0 1 4 2 3 0 1 3 4 2 1 3 1 4 1 1 3 0 1 1 4 3 4 2 2 3 1 0 1 2 1 3 3 3 4
 2 3 1 1 2 1 4]


In [18]:
def accuracy(predictions, target):
    """Calculates total accuracy for our model.
    
    Args- 
        predictions- Predicted target outcomes as predicted by our MLR function
        target- Actual target values
    
    Returns-
        accuracy- Accuracy percentage of our model
    """
    correctPred = 0
    n = predictions.shape[0]
    for i in range(n):
        if predictions[int(i)] == target[int(i)]:
            correctPred += 1
    accuracy = correctPred/n * 100
    return accuracy

accuracy = accuracy(predictions, target) #calculating accuracy for our model
print(accuracy)

16.5016501650165


Model Optimization

In [19]:
def train_test_split(dataframe, test_size = 0.2):
    """Splits dataset into training and testing sets.
    
    Args- 
        dataframe- The dataframe object you want to split
        test_size- Size of test dataset that you want
    
    Returns-
        train_features, train_target, test_features, test_target 
    """
    
    data = dataframe.to_numpy() # converts dataframe to numpy array
    totalRows = data.shape[0] # total rows in the dataset
    testRows = np.round(totalRows * test_size) # total rows in testing dataset
    randRowNum = np.random.randint(0, int(totalRows), int(testRows)) # randomly generated row numbers
    testData = np.array([data[i] for i in randRowNum]) # creates test dataset
    data = np.delete(data, randRowNum, axis = 0) # deletes test data rows from main dataset; making it training dataset
    train_features = data[:, :-1]
    train_target = data[:, -1].astype(int)
    test_features = testData[:, :-1]
    test_target = testData[:, -1].astype(int)
    
    return train_features, train_target, test_features, test_target    

# running train_test_split for our dataset
train_features, train_target, test_features, test_target = train_test_split(df, test_size = 0.17)
standardScaler(train_features) # standard scaling training set 
standardScaler(test_features) # standard scaling testing set
train_features.shape, train_target.shape, test_features.shape, test_target.shape

((258, 13), (258,), (52, 13), (52,))

In [21]:
# creating one-hot-coded target labels for training target labels
oneHotTraining = np.array([np.zeros(5) for i in range(train_target.shape[0])])
for label,i in zip(oneHotTraining,train_target):
    label[int(i)] = 1
    
# creating one-hot-coded target labels for testing target labels
oneHotTest = np.array([np.zeros(5) for i in range(test_target.shape[0])])
for label,i in zip(oneHotTest,test_target):
    label[int(i)] = 1

In [23]:
def crossEntropyLoss(probabilities, target):
    """Calculates cross entropy loss for a set of predictions and actual targets.
    
    Args-
        predictions- Probability predictions, as returned by multinomialLogReg function
        target- Actual target values
    Returns- 
        CELoss- Average cross entropy loss
    """
    n_samples = probabilities.shape[0]
    CELoss = 0
    for sample, i in zip(probabilities, target):
        CELoss += -np.log(sample[i])
    CELoss /= n_samples
    return CELoss  

In [25]:
crossEntropyLoss(probabilities, target)

2.0429148583747136

In [27]:
def stochGradDes(learning_rate, epochs, target, features, weights, biases):
    """Performs stochastic gradient descent optimization on the model.
    
    Args-
        learning_rate- Size of the step the function will take during optimization
        epochs- No. of iterations the function will run for on the model
        target- Numpy array containing actual target values
        features- Numpy array of independent variables
        weights- Numpy array containing weights associated with each feature
        biases- Array containinig model biases
    
    Returns-
        weights, biases, loss_list
        where,
            weights- Latest weight calculated (Numpy array)
            bias- Latest bias calculated (Numpy array)
            loss_list- Array containing list of losses observed after each epoch    
    """
    target = target.astype(int)
    loss_list = np.array([]) #initiating an empty array
    
    for i in range(epochs):
        probabilities, _ = multinomialLogReg(features, weights, biases) # Calculates probabilities for each possible outcome
        
        CELoss = crossEntropyLoss(probabilities, target) # Calculates cross entropy loss for actual target and predictions
        loss_list = np.append(loss_list, CELoss) # Adds the CELoss value for the epoch to loss_list
        
        probabilities[np.arange(features.shape[0]),target] -= 1 # Substract 1 from the scores of the correct outcome
        
        grad_weight = probabilities.T.dot(features) # gradient of loss w.r.t. weights
        grad_biases = np.sum(probabilities, axis = 0).reshape(-1,1) # gradient of loss w.r.t. biases
        
        #updating weights and biases
        weights -= (learning_rate * grad_weight)
        biases -= (learning_rate * grad_biases)
        
    return weights, biases, loss_list

In [29]:
updatedWeights, updatedBiases, loss_list = stochGradDes(0.1, 2000, train_target, train_features, weights, biases)

In [31]:
updatedWeights

array([[  0.16587783,  -1.74108124,  -1.34940154,  -2.17044221,
         -0.29361831,   4.33564021,  -2.42687728,   1.99313757,
         -0.85884515,  -2.44358111,  -1.73055392,  -4.30319062,
         -3.85399803],
       [  1.49067559,   2.92299884,   0.55640214,   0.30205324,
          1.93989756,   0.28287286,   2.11665973,   1.81769515,
         -1.45925298,  -3.24226449,  -1.50888411,  -1.83650378,
          1.75222193],
       [  0.03672809,   0.11423268,   1.57716571,   0.63514804,
          1.42543618,   4.21314323,  -0.59313507,  -1.10409271,
          0.54574436,   1.87051865,   1.17083727,   1.43944521,
          0.3758755 ],
       [ -2.76902572,  -0.96773596,   0.06752916,   1.82926821,
          0.23516821,   4.13537549,   1.24803419,  -2.61610418,
          1.00284578,   1.89267438,   0.09424301,   3.34619185,
          2.13366299],
       [  3.3139834 ,   2.10159796,   1.88070475,   2.10084354,
          0.80617048, -10.74139508,   2.04653378,   2.19190888,
          3.

In [34]:
testProbabilities, testPredictions = multinomialLogReg(test_features, updatedWeights, updatedBiases)

correctPreds = 0
for i in range(len(testPredictions)):
    if testPredictions[i] == test_target[i]:
        correctPreds += 1
acc = correctPreds / len(testPredictions) * 100
print("Model accuracy on test dataset - {}".format(acc))

Model accuracy on test dataset - 34.61538461538461
